# Homework 2: Social Network Data Visualisation

- [Physics as network](#Physics as network)
- [Getting the Data](#Getting the Data)
- [Cleaning the data](#Cleaning the data)
- [Compiling the nodes data](#Compiling the nodes data)
- [Compiling the links data](#Compiling the links data)

In this homework, we are going to reuse the data set of the previous homework to look at it from a Social Network prospective. A social network is a construct where social actors can be represented as nodes linked by edges on a graph. As an example, here is the Facebook social network

<img src="additional/facebook_world_friend_map.png",width=600>

We are using the [d3.js](https://d3js.org/) javascript library, a data visualization library, to represent our networks. You can see [here](https://bl.ocks.org/mbostock/4062045) an example of such a network.


## Physics as a social network  <a class="anchor" id="Physics as network"></a>

A social network can be represented as a graph with a set of nodes and a set of links between those nodes:

<img src="additional/small_undirected_network_labeled.png",width=300>

In the previous homework, we used data sets with physicists and physics domains. If we consider each physicist and each physics domain as a possible node on a network, the problem becomes building edges between those. Here an example of how to represent graphs with python data structures:

In [3]:
cd /home/jay/Documents/notebooks/Homework2

/home/jay/Documents/notebooks/Homework2


In [80]:
small_network = {
    "nodes": [
        {"id": "Albert Einstein"},
        {"id": "Paul Dirac"},
        {"id": "Niels Bohr"}
    ],
    "links": [
        {"source": "Albert Einstein", "target": "Paul Dirac"},
        {"source": "Albert Einstein", "target": "Niels Bohr"},
        {"source": "Paul Dirac", "target": "Niels Bohr"}
    ]
}

## We dump this network into a .json file
import json
with open("small_network.json","w") as f:
    json.dump(small_network, f, indent=4)

If you have a mac, the following script is going to open a safari window to visualize this network, otherwise you can just open the `small_network.html` file with Safari or Firefox. For some reason, it does not work with chrome. 

In [2]:
import os
os.system("open -a /Applications/Safari.app ./small_network.html")

256

Obviously, that is not a very interesting network and we are going to build a more substantial one! D3.js expects the data in a specific format as shown above and we are going to shape our data to follow those requirements. For each node, we need an "id" tag and we can add other attribute if we desire. For the links, we need a "source" and a "target" to connect nodes and we can add other attributes also. 

In the following network, we are going to add the "length" attribute that captures how big is each node and the "value" attribute for each link that captures how "strong" the links are. We also are going to distinguish between 2 types of nodes: the nodes for the physicist and the nodes for the physics domains. Those 2 sets of nodes will be distinguished by the attribute "group". For example:

```
small_network = {
    "nodes": [
        {"id": "Albert Einstein", "group": 1, "length": 100},
        {"id": "Paul Dirac", "group": 1, "length": 200},
        {"id": "Niels Bohr", "group": 1, "length": 300}
    ],
    "links": [
        {"source": "Albert Einstein", "target": "Paul Dirac", "value": 0.5},
        {"source": "Albert Einstein", "target": "Niels Bohr", "value": 0.4},
        {"source": "Paul Dirac", "target": "Niels Bohr", "value": 0.3}
    ]
}
```

## Getting the Data <a class="anchor" id="Getting the Data"></a>

We first going to gather the data needed for this project. We are going to extract the words in each Wikipedia page to understand the relation between each physicist and physics domain. 

In [8]:
## We get the nobel data set
import numpy as np
import pandas as pd
from httplib2 import Http
from bs4 import BeautifulSoup, SoupStrainer

class Parser:
    
    def __init__(self, url):  
        http = Http()
        status, response = http.request(url)
        tables = BeautifulSoup(response, "lxml", 
                              parse_only=SoupStrainer("table", {"class":"wikitable sortable"}))
        self.table = tables.contents[1]
    
    def parse_table(self):      
        rows = self.table.find_all("tr")
        header = self.parse_header(rows[0])
        table_array = [self.parse_row(row) for row in rows[1:]]
        table_df = pd.DataFrame(table_array, columns=header).apply(self.clean_table, 1)
        return table_df.replace({"Year":{'':np.nan}})
        
    def parse_row(self, row):     
        columns = row.find_all("td")
        return [BeautifulSoup.get_text(col).strip() for col in columns if BeautifulSoup.get_text(col) != '']
    
    def parse_header(self, row):     
        columns = row.find_all("th")
        return [BeautifulSoup.get_text(col).strip() for col in columns if BeautifulSoup.get_text(col) != ""]
    
    def clean_table(self, row):
        if not row.iloc[0].isdigit() and row.iloc[0] != '':
            return row.shift(1)
        else:
            return row
        
url = "https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics"        
parser = Parser(url)   
nobel_df = parser.parse_table()
nobel_df.columns = ["Year", "Laureate", "Country", "Rationale"]
nobel_df.dropna(subset=["Country"], inplace=True)
nobel_df.fillna(method="ffill", inplace=True)
nobel_df.drop(["Year", "Country", "Rationale"], 1, inplace=True)

http = Http()
status, response = http.request(url)

table = BeautifulSoup(response, "lxml", parse_only=SoupStrainer('table'))
link_df = pd.DataFrame([[x.string, x["href"]] for x in table.contents[1].find_all("a")],
                       columns=["Laureate", "link"]).drop_duplicates()

nobel_df = nobel_df.merge(link_df, on="Laureate", how="left")
nobel_df.set_index("Laureate", inplace=True)
nobel_df.drop_duplicates(inplace=True)
nobel_df

,link
Laureate,
Wilhelm Conrad Röntgen,/wiki/Wilhelm_R%C3%B6ntgen
Hendrik Lorentz,/wiki/Hendrik_Lorentz
Pieter Zeeman,/wiki/Pieter_Zeeman
Antoine Henri Becquerel,/wiki/Henri_Becquerel
Pierre Curie,/wiki/Pierre_Curie
Maria Skłodowska-Curie,/wiki/Maria_Sk%C5%82odowska-Curie
Lord Rayleigh,"/wiki/John_Strutt,_3rd_Baron_Rayleigh"
Philipp Eduard Anton von Lenard,/wiki/Philipp_Lenard
Joseph John Thomson,/wiki/J._J._Thomson


We also going to extract the links of each of the physics domains listed in the Research fields table of the [https://en.wikipedia.org/wiki/Physics](https://en.wikipedia.org/wiki/Physics) Wikipedia page.

In [9]:
## We get the physics links
url = "https://en.wikipedia.org/wiki/Physics"

http = Http()
status, response = http.request(url)

table = BeautifulSoup(response, "lxml", parse_only=SoupStrainer('table'))
physics_df = pd.DataFrame([[x.string.lower(), x["href"].lower()] for x in table.contents[2].find_all("a")],
                       columns=["Physics_domain", "link"]).drop_duplicates()

physics_df = physics_df.groupby("Physics_domain").first()
physics_df

,link
Physics_domain,
accelerator physics,/wiki/accelerator_physics
acoustics,/wiki/acoustics
agrophysics,/wiki/agrophysics
antimatter,/wiki/antimatter
applied physics,/wiki/applied_physics
astrometry,/wiki/astrometry
astronomy,/wiki/astronomy
astrophysics,/wiki/astrophysics
atom,/wiki/atom



## Cleaning the data <a class="anchor" id="Cleaning the data"></a>

>Use the code from your previous homework to create the different functions to clean the get the word data and clean it.

In [10]:
from string import punctuation

## We get the bios
def get_text(link, root_website = "https://en.wikipedia.org"):    
    http = Http()
    status, response = http.request(root_website + link)
    body = BeautifulSoup(response, "lxml", parse_only=SoupStrainer("div", {"id":"mw-content-text"}))
    return BeautifulSoup.get_text(body.contents[1])

# TODO: copy your clean_string function from the previous homework
def clean_string(string):
    for p in punctuation + "1234567890":
        string = string.replace(p,'').lower()   
    return string

# TODO: copy your remove function from the previous homework
def remove(list_to_clean, element_to_remove=[None, ""]):
    return list(filter(lambda x: x != element_to_remove, list_to_clean))

# TODO: copy your remove_one function from the previous homework
def remove_one(list_to_clean):
    return list(filter(lambda x: len(x) > 1, list_to_clean))


We now going to write a function that takes a data frame with a "link" column and return a column of list of words. We basically aggregate all the above function into one to reproduce what was done in the previous homework. Note that here we DO NOT use the function that keep only a unique element of each list nor the one that filter on the number of occurance.

> Write a function that applies all the previous functions to clean a text.

In [11]:
from nltk.corpus import stopwords
words_to_remove = set(stopwords.words('english'))

#remove stopwords
def remove_stopword(list_to_clean, remove_list = words_to_remove):
    return list(filter(lambda x: x not in remove_list, list_to_clean))

# TODO: aggregate all the above function into one to return a list of words from each link
def clean_everything(df):
    return df['link'].apply(get_text).apply(clean_string).str.split().apply(remove).apply(remove_stopword).apply(remove_one)

    
    
physics_df["physics_list"] = clean_everything(physics_df)
nobel_df["physics_list"] = clean_everything(nobel_df)
nobel_df

,link,physics_list
Laureate,,
Wilhelm Conrad Röntgen,/wiki/Wilhelm_R%C3%B6ntgen,"[wilhelm, röntgen, born, wilhelm, conrad, rönt..."
Hendrik Lorentz,/wiki/Hendrik_Lorentz,"[confused, hendrikus, albertus, lorentz, ludvi..."
Pieter Zeeman,/wiki/Pieter_Zeeman,"[pieter, zeeman, born, may, zonnemaire, nether..."
Antoine Henri Becquerel,/wiki/Henri_Becquerel,"[uses, see, becquerel, disambiguation, antoine..."
Pierre Curie,/wiki/Pierre_Curie,"[pierre, curie, born, may, paris, france, died..."
Maria Skłodowska-Curie,/wiki/Maria_Sk%C5%82odowska-Curie,"[article, polish, physicist, uses, see, marie,..."
Lord Rayleigh,"/wiki/John_Strutt,_3rd_Baron_Rayleigh","[lord, rayleigh, om, prs, born, november, lang..."
Philipp Eduard Anton von Lenard,/wiki/Philipp_Lenard,"[waterfall, effect, redirects, illusory, visua..."
Joseph John Thomson,/wiki/J._J._Thomson,"[article, nobel, laureate, physicist, moral, p..."


We saw last time that there are many words that are not relevant to physics concepts in those Wikipedia pages. We are going to attempt to filter those with the simple following approach. 

We are going to compile a set of all the unique words in the `nobel_df` lists and a set of all the unique words in the `physics_df` lists. By taking the intersection of those 2 sets, we can subset the words corpus to something more relevant to physics.

>- Compile a set of all unique words in `nobel_df["physics_list"]`. You can use the function [`pd.sum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sum.html) to concatenate lists. You can cast the final list to a [`set`](https://docs.python.org/2/library/sets.html)
- Compile a set of all unique words in `physics_df["physics_list"]`.
- Compile the intersection of those 2 sets using the `intersection` function.

In [12]:
physics_df['physics_list'].sum()

['accelerator',
 'physics',
 'branch',
 'applied',
 'physics',
 'concerned',
 'designing',
 'building',
 'operating',
 'particle',
 'accelerators',
 'described',
 'study',
 'motion',
 'manipulation',
 'observation',
 'relativistic',
 'charged',
 'particle',
 'beams',
 'interaction',
 'accelerator',
 'structures',
 'electromagnetic',
 'fields',
 'also',
 'related',
 'fields',
 'microwave',
 'engineering',
 'accelerationdeflection',
 'structures',
 'radio',
 'frequency',
 'range',
 'optics',
 'emphasis',
 'geometrical',
 'optics',
 'beam',
 'focusing',
 'bending',
 'laser',
 'physics',
 'laserparticle',
 'interaction',
 'computer',
 'technology',
 'emphasis',
 'digital',
 'signal',
 'processing',
 'eg',
 'automated',
 'manipulation',
 'particle',
 'beam',
 'experiments',
 'conducted',
 'particle',
 'accelerators',
 'regarded',
 'part',
 'accelerator',
 'physics',
 'belong',
 'according',
 'objectives',
 'experiments',
 'eg',
 'particle',
 'physics',
 'nuclear',
 'physics',
 'condensed',


In [13]:
# TODO: find all the words in nobel_df["physics_list"]
all_nobel_words =  set(nobel_df['physics_list'].sum())

# TODO: find all the words in physics_df["physics_list"]
all_physics_words =  set(physics_df['physics_list'].sum())

# TODO: find all the intersection of all_nobel_words and all_physics_words
physics_corpus =  set(all_nobel_words.intersection(all_physics_words))

physics_corpus

{'ampère',
 'copper',
 'routine',
 'sep',
 'lettere',
 'varying',
 'governed',
 'michel',
 'thicker',
 'heikki',
 'alaska',
 'wherein',
 'mark',
 'universitys',
 'bibcodephrvp',
 'joos',
 'coordinate',
 'bloomberg',
 'transformer',
 'items',
 'law',
 'msu',
 'popularly',
 'columbia',
 'systematically',
 'excluded',
 'kox',
 'suppression',
 'da',
 'lr',
 'doitf',
 'tamara',
 'demidov',
 'hugh',
 'unwin',
 'hose',
 'course',
 'tanaka',
 'dedicated',
 'semiclassical',
 'organic',
 'plot',
 'netherlands',
 'belief',
 'kurt',
 'doi',
 'capsule',
 'legislative',
 'altogether',
 'catalogue',
 'even',
 'vieweg',
 'raysedit',
 'rudolph',
 'alwyn',
 'rahman',
 'dictionary',
 'elsewhere',
 'ricard',
 'roald',
 'dispersed',
 'jennifer',
 'florence',
 'perform',
 'fictional',
 'certainly',
 'oscillating',
 'reducing',
 'superconductivityedit',
 'endless',
 'brightness',
 'philosophy',
 'distributions',
 'check',
 'pomeranchuk',
 'notices',
 'outline',
 'orange',
 'gabriele',
 'acceptor',
 'outcome'

In [14]:
print(len(physics_corpus), len(all_nobel_words), len(all_physics_words))

12619 31053 33483


>Write a function that keep only specific words from a list

In [15]:
nobel_df['physics_list']

Laureate
Wilhelm Conrad Röntgen             [wilhelm, röntgen, born, wilhelm, conrad, rönt...
Hendrik Lorentz                    [confused, hendrikus, albertus, lorentz, ludvi...
Pieter Zeeman                      [pieter, zeeman, born, may, zonnemaire, nether...
Antoine Henri Becquerel            [uses, see, becquerel, disambiguation, antoine...
Pierre Curie                       [pierre, curie, born, may, paris, france, died...
Maria Skłodowska-Curie             [article, polish, physicist, uses, see, marie,...
Lord Rayleigh                      [lord, rayleigh, om, prs, born, november, lang...
Philipp Eduard Anton von Lenard    [waterfall, effect, redirects, illusory, visua...
Joseph John Thomson                [article, nobel, laureate, physicist, moral, p...
Albert Abraham Michelson           [confused, athlete, albert, michelsen, albert,...
Gabriel Lippmann                   [gabriel, lippmann, born, jonas, ferdinand, ga...
Guglielmo Marconi                  [marconi, redirects, 

In [16]:
# TODO: write a function that keep only specific words from a list
def keep_only(list_to_clean, corpus=physics_corpus):
    return list(filter(lambda x: x in corpus, list_to_clean))
    
nobel_df["physics_list_clean"] = nobel_df["physics_list"].apply(keep_only)
physics_df["physics_list_clean"] = physics_df["physics_list"].apply(keep_only)

In [17]:
physics_df["physics_list_clean"]

Physics_domain
accelerator physics                       [accelerator, physics, branch, applied, physic...
acoustics                                 [uses, see, acoustics, disambiguation, artific...
agrophysics                               [article, includes, list, references, related,...
antimatter                                [uses, see, antimatter, disambiguation, antima...
applied physics                           [see, also, outline, applied, physics, enginee...
astrometry                                [illustration, use, interferometry, optical, w...
astronomy                                 [article, scientific, study, celestial, object...
astrophysics                              [article, use, physics, chemistry, determine, ...
atom                                      [uses, see, atom, disambiguation, helium, atom...
atomic and molecular astrophysics         [within, million, years, light, bright, stars,...
atomic physics                            [article, includes, lis


## Compiling the nodes data <a class="anchor" id="Compiling the nodes data"></a>

For those 2 dataframes, we are going to create 2 additional columns:
    
>- create columns "length" that counts the number of words in each list. This column will be used to capture the size of the nodes in the networks. Basically we are going to say: the more words in the Wikipedia page, the more significant the physicist or physics domain is.
- create columns "group" with a unique value for each of those dataframes. Set the value to 1 in the `nobel_df` dataframe and 0 for the `physics_df` dataframe. This columns will be used to distinguish the physicists from the physics domains and attribute them different colors in the network visualization.

In [18]:
# TODO: compute the length of each list
nobel_df["length"] =  nobel_df['physics_list_clean'].apply(len)
physics_df["length"] =  physics_df['physics_list_clean'].apply(len)

# TODO: Set this column to 1
nobel_df["group"] =  1
# TODO: Set this column to 0
physics_df["group"] =  0

In [19]:
physics_df

,link,physics_list,physics_list_clean,length,group
Physics_domain,,,,,
accelerator physics,/wiki/accelerator_physics,"[accelerator, physics, branch, applied, physic...","[accelerator, physics, branch, applied, physic...",853,0
acoustics,/wiki/acoustics,"[uses, see, acoustics, disambiguation, artific...","[uses, see, acoustics, disambiguation, artific...",2047,0
agrophysics,/wiki/agrophysics,"[article, includes, list, references, related,...","[article, includes, list, references, related,...",400,0
antimatter,/wiki/antimatter,"[uses, see, antimatter, disambiguation, antima...","[uses, see, antimatter, disambiguation, antima...",3342,0
applied physics,/wiki/applied_physics,"[see, also, outline, applied, physics, enginee...","[see, also, outline, applied, physics, enginee...",283,0
astrometry,/wiki/astrometry,"[illustration, use, interferometry, optical, w...","[illustration, use, interferometry, optical, w...",1133,0
astronomy,/wiki/astronomy,"[article, scientific, study, celestial, object...","[article, scientific, study, celestial, object...",5169,0
astrophysics,/wiki/astrophysics,"[article, use, physics, chemistry, determine, ...","[article, use, physics, chemistry, determine, ...",1958,0
atom,/wiki/atom,"[uses, see, atom, disambiguation, helium, atom...","[uses, see, atom, disambiguation, helium, atom...",6602,0


Let's concatenate the those 2 dataframes into the `nodes_df` dataframe. 

>Use the [`pd.concat`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) function to do so and only keep the "length" and "group" columns. The concatenation needs to be done along the row axis.

In [20]:
nobel_df[['length', 'group']]

,length,group
Laureate,,
Wilhelm Conrad Röntgen,1245,1
Hendrik Lorentz,2710,1
Pieter Zeeman,984,1
Antoine Henri Becquerel,1156,1
Pierre Curie,1428,1
Maria Skłodowska-Curie,4941,1
Lord Rayleigh,1410,1
Philipp Eduard Anton von Lenard,1242,1
Joseph John Thomson,2889,1


In [21]:
# TODO: concatenate those two dataframe into the nodes_df dataframe. 
# keep only the "length" and "group" columns.
nodes_df =   pd.concat([nobel_df[['length', 'group']], physics_df[['length','group']]], axis=0)

nodes_df.index.name = "id"
nodes_df

,length,group
id,,
Wilhelm Conrad Röntgen,1245,1
Hendrik Lorentz,2710,1
Pieter Zeeman,984,1
Antoine Henri Becquerel,1156,1
Pierre Curie,1428,1
Maria Skłodowska-Curie,4941,1
Lord Rayleigh,1410,1
Philipp Eduard Anton von Lenard,1242,1
Joseph John Thomson,2889,1


From this dataframe, we can easily format the data as a list of dictionaries as the d3.js library expects the data to be. We have the "length" attribute for the size of the node, the "group" attribute to distinguish between physicists and physics domains and each node has a unique "id" tag represented by the names.  

In [22]:
nodes_list = list(nodes_df.reset_index().transpose().to_dict().values())
nodes_list

[{'group': 1, 'id': 'Wilhelm Conrad Röntgen', 'length': 1245},
 {'group': 1, 'id': 'Hendrik Lorentz', 'length': 2710},
 {'group': 1, 'id': 'Pieter Zeeman', 'length': 984},
 {'group': 1, 'id': 'Antoine Henri Becquerel', 'length': 1156},
 {'group': 1, 'id': 'Pierre Curie', 'length': 1428},
 {'group': 1, 'id': 'Maria Skłodowska-Curie', 'length': 4941},
 {'group': 1, 'id': 'Lord Rayleigh', 'length': 1410},
 {'group': 1, 'id': 'Philipp Eduard Anton von Lenard', 'length': 1242},
 {'group': 1, 'id': 'Joseph John Thomson', 'length': 2889},
 {'group': 1, 'id': 'Albert Abraham Michelson', 'length': 2227},
 {'group': 1, 'id': 'Gabriel Lippmann', 'length': 1472},
 {'group': 1, 'id': 'Guglielmo Marconi', 'length': 4077},
 {'group': 1, 'id': 'Karl Ferdinand Braun', 'length': 832},
 {'group': 1, 'id': 'Johannes Diderik van der Waals', 'length': 2187},
 {'group': 1, 'id': 'Wilhelm Wien', 'length': 741},
 {'group': 1, 'id': 'Nils Gustaf Dalén', 'length': 778},
 {'group': 1, 'id': 'Heike Kamerlingh-Onne


## Compiling the links data <a class="anchor" id="Compiling the links data"></a>

We have the nodes, we need to find a way to connect them. We are going to compute the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) between each of the wikipedia pages. It is called the cosine similarity because a dot product between 2 vectors $\mathbf{A}$ and $\mathbf{B}$ can be express as:
\begin{equation}
A\cdot B = \Vert A\Vert_2\Vert B\Vert_2\cos\theta
\end{equation}
where $\theta$ is the angle between the 2 vectors. Similarly:
\begin{equation}
\cos\theta = \frac{ A\cdot B}{\Vert A\Vert_2\Vert B\Vert_2}
\end{equation}
$\cos\theta\in[-1,1]$ and specifically $\cos\theta = 1$ if the 2 vectors are in the same direction or $\cos\theta = -1$ if the 2 vectors are in the opposite direction. The matter here becomes to be able to express a Wikipedia page as a vector. I suggest here a simple approach but there are many ways to achieve this. 

We defined earlier the corpus of physics words `physics_corpus`. Each word can be thought as a orthogonal basis defining a vector space where our Wikipedia pages are living in. Each component can be represented by the number of time a specific word appear in a page. As an example, imaging a page $P$ represented by the following list of words:
```
P_list = ["data", "data", "science", "python", "python"]
```
And let's imaging that we have a simple word corpus:
```
corpus = {"engineering", "data", "science", "python"}
```
Then in that basis $P$ could be represented by a vector 
\begin{equation}
\mathbf{P}=\left(\begin{matrix}
  0  \\
  2  \\
  1  \\
  2
 \end{matrix}\right)
\end{equation}

We need to express each Wikipedia page as such vector. 

>Let's start by creating a dataframe with as columns, all the indices of the `nodes_df` dataframe (`nodes_df.index.values`) and as index, the whole `physics_corpus` set.

In [23]:
# TODO: create a data frame with the index of nodes_df as columns and physics_corpus as index
words_vector =  pd.DataFrame(index=physics_corpus, columns = nodes_df.index.values)
words_vector

,Wilhelm Conrad Röntgen,Hendrik Lorentz,Pieter Zeeman,Antoine Henri Becquerel,Pierre Curie,Maria Skłodowska-Curie,Lord Rayleigh,Philipp Eduard Anton von Lenard,Joseph John Thomson,Albert Abraham Michelson,...,superfluid,supernova,superstring theory,supersymmetry,surface physics,theory of everything,universe,vacuum energy,vehicle dynamics,weak
ampère,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
copper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
routine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lettere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
varying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
governed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
michel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
thicker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heikki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To fill this table we are going to use the [`value_counts`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) function on the list of words contained in `nobel_df["physics_list_clean"]` and `physics_df["physics_list_clean"]`. 

>Write a function that takes a list and return a value_counts. The return value should be a pandas series with the words as index. We are using this function to populate the `words_vector` dataframe. Note that because `words_vector` already has an index, the values get populated at the right place automatically.

In [24]:
from collections import Counter
Counter(nobel_df["physics_list_clean"].sum())

Counter({'wilhelm': 139,
         'röntgen': 273,
         'born': 987,
         'conrad': 14,
         'march': 469,
         'province': 13,
         'german': 383,
         'died': 320,
         'february': 399,
         'germany': 249,
         'alma': 200,
         'mater': 200,
         'zurich': 44,
         'university': 2917,
         'known': 472,
         'discovery': 393,
         'xrays': 63,
         'awards': 458,
         'medal': 979,
         'elliott': 33,
         'nobel': 2506,
         'prize': 2523,
         'physics': 3824,
         'scientific': 738,
         'career': 405,
         'fields': 290,
         'xray': 83,
         'astronomy': 135,
         'institutions': 208,
         'würzburg': 15,
         'doctoral': 282,
         'advisor': 176,
         'august': 359,
         'students': 256,
         'herman': 57,
         'ioffe': 8,
         'ernst': 74,
         'wagner': 6,
         'rudolf': 95,
         'ladenburg': 4,
         'notable': 68,
      

In [25]:
#TODO: write a function that take a list and return the a word count
def count_words(list_to_count):
    return pd.Series(list_to_count).value_counts()

words_vector.loc[:,nobel_df.index] = nobel_df["physics_list_clean"].apply(count_words).transpose()
words_vector.loc[:,physics_df.index] = physics_df["physics_list_clean"].apply(count_words).transpose()
words_vector

,Wilhelm Conrad Röntgen,Hendrik Lorentz,Pieter Zeeman,Antoine Henri Becquerel,Pierre Curie,Maria Skłodowska-Curie,Lord Rayleigh,Philipp Eduard Anton von Lenard,Joseph John Thomson,Albert Abraham Michelson,...,superfluid,supernova,superstring theory,supersymmetry,surface physics,theory of everything,universe,vacuum energy,vehicle dynamics,weak
ampère,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
copper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
routine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lettere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
varying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1
governed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
michel,NaN,NaN,NaN,1,NaN,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
thicker,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heikki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


There are many entries in this dataframe that appear as `NaN`. We just need to replace those missing values by 0 since they indicate that no records of the word were found for those pages. 

>Use the function [`pd.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) to fill with 0.

In [26]:
# TODO: fill the missing values
words_vector = words_vector.fillna(0)
words_vector

,Wilhelm Conrad Röntgen,Hendrik Lorentz,Pieter Zeeman,Antoine Henri Becquerel,Pierre Curie,Maria Skłodowska-Curie,Lord Rayleigh,Philipp Eduard Anton von Lenard,Joseph John Thomson,Albert Abraham Michelson,...,superfluid,supernova,superstring theory,supersymmetry,surface physics,theory of everything,universe,vacuum energy,vehicle dynamics,weak
ampère,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
copper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
routine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sep,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lettere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
varying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
governed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
michel,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
thicker,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
heikki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


>- Write a function that takes 2 vectors (2 pandas series) and return the cosine similarity index. You can use the function [`dot`](http://pandas.pydata.org/pandas-docs/version/0.18.1/generated/pandas.Series.dot.html), [`pow`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pow.html) and `sum` if you like.
- Use this function to fill the `similarity_df` dataframe
- Bonus points if you can compute this dataframe using matrix algebra ([`dot`](http://pandas.pydata.org/pandas-docs/version/0.18.1/generated/pandas.DataFrame.dot.html)) without having to iterate through the columns. Hint create 2 dataframe: one that is the dot products of words_vector with itself and one that represent a matrix of norm products. Then divide one matrix by the other element wise. For this case, you would not need to use the `compute_similarity` function.

In [63]:
words_vector.iloc[:,0]

ampère                  0.0
copper                  0.0
routine                 0.0
sep                     0.0
lettere                 0.0
varying                 0.0
governed                0.0
michel                  0.0
thicker                 1.0
heikki                  0.0
alaska                  0.0
wherein                 0.0
mark                    0.0
universitys             0.0
bibcodephrvp            0.0
joos                    0.0
coordinate              0.0
bloomberg               0.0
transformer             0.0
items                   0.0
law                     0.0
msu                     0.0
popularly               0.0
columbia                1.0
systematically          0.0
excluded                0.0
kox                     0.0
suppression             0.0
da                      0.0
lr                      0.0
                       ... 
mits                    0.0
ubc                     0.0
related                 4.0
bottom                  0.0
fluorescence        

In [72]:
words_vector.apply(np.linalg.norm)

Wilhelm Conrad Röntgen              85.819578
Hendrik Lorentz                    177.943811
Pieter Zeeman                       76.288924
Antoine Henri Becquerel             72.456884
Pierre Curie                       108.940351
Maria Skłodowska-Curie             338.911493
Lord Rayleigh                       98.040808
Philipp Eduard Anton von Lenard     75.139870
Joseph John Thomson                188.109011
Albert Abraham Michelson           119.879106
Gabriel Lippmann                    81.055537
Guglielmo Marconi                  196.629092
Karl Ferdinand Braun                59.295868
Johannes Diderik van der Waals     182.742989
Wilhelm Wien                        50.507425
Nils Gustaf Dalén                   40.693980
Heike Kamerlingh-Onnes             100.264650
Max von Laue                       214.387500
William Henry Bragg                117.826143
William Lawrence Bragg              92.168324
Charles Glover Barkla               36.124784
Max Planck                        

In [77]:
mag_df.shape

()

In [83]:
# TODO: write a function that takes 2 vectors and return the cosine similarity index
def compute_similarity(vect1, vect2):
    return vect1.dot(vect2) / (np.sqrt(vect1.pow(2).sum()) * np.sqrt(vect2.pow(2).sum()))

similarity_df = pd.DataFrame(columns=words_vector.columns, index=words_vector.columns, dtype=float)

# TODO: fill the similarity_df dataframe with the cosine similarity
dot_df = np.dot(words_vector.T, words_vector)
mag = words_vector.apply(np.linalg.norm)
mag_df = np.dot(mag, mag.T)

cosine = np.divide(dot_df, mag_df)
# TODO: bonus points if you can compute this dataframe using matrix algebra 

for i in range(336):
    for j in range(336):
        similarity_df.iloc[i,j] = cosine[i][j]



#similarity_df.iloc[i,j] = cosine[i][j] for i in range(336) for j in range(336)
cosine

print(type(mag))
mag

Exception: Data must be 1-dimensional

We are now going to "melt" `similarity_df` into a long dataframe: 

>- We need to reset the index of `similarity_df` ([`pd.reset_index`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html))
- Then we use the [`melt`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) function to melt the dataframe. Use the resetted index as `id_vars`. 

In [ ]:
# TODO: reset the index and melt the dataframe
melted_df =  # YOUR CODE

melted_df.columns = ["source", "target", "value"]
melted_df

You can see that at this point we have something close to what is necessary to create the links data. There are 3 things we need do to finalize our dataset. First, it is unnecessary to have "source" equal to the "target" (a node does not need to be linked to itself). Second, we have a duplicated links because in our case a "source" has the same role than a "target" and can be interchanged (our graph is not directed). Third, we need to subset the links set because there are too many for the program to run efficiently.

Let's shuffle the data set rowwise ([`sample`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html)). This help us to not bias our links selection due to a prior alphabetic ordering of the data

In [ ]:
melted_df = melted_df.sample(frac=1.).reset_index(drop=True)
melted_df

We then going to find the pairs of ("source", "target") that are equal to pairs ("target", "source"). To do that we are going to merge `melted_df` with itself where ("source", "target") = ("target", "source").

>- merge it with itself with `left_on=["source", "target"]` and `right_on=["target", "source"]`. Pass the dataframe with the index resetted using [`pd.reset_index`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html).

In [ ]:
# TODO: merge melted_df with itself
merged_df =  # YOUR CODE

merged_df

At this point, we can see that each pair of ("source", "target") has the redondant equivalent ("target", "source"). This also highlight the cases where "source" = "target". To filter the useless rows we can simply pick the ("source", "target") pair or the ("target", "source") pick to remove. Let's choose which pair to remove by capturing the index we want to remove

>- Look at the pair of columns `merged_df[["index_x", "index_y"]]` and simple choose the greater between the two using [`max`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.max.html). By selecting only the unique values ([`unique`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html)) of the resulting list of indices we have selected the index to remove and we can drop them using [`drop`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html). 

In [ ]:
# TODO: find the index to drop
index_to_drop =  # YOUR CODE
# TODO: use the index_to_drop to subset the melted_df dataframe
melted_df_sub =  # YOUR CODE
melted_df_sub

We have filtered quite a bit of rows but it still is too many for the network simulation to run efficiently. For each source, we are going to select the 10 highest values. 

>- Group `melted_df_sub` by "source" using the [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) method and select the 10 targets that have the highest values using the [`nlargest`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nlargest.html) method.
- The resulting pandas Series has a multiindex with 2 levels. We need to get the level 1 of the multiindex to know which rows to keep in `melted_df_sub`. You can get it using the function [`get_level_values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Index.get_level_values.html) on the index. 

In [ ]:
# TODO: Group melted_df_sub by "source" using the groupby method and select the 10 
# targets that have the highest values using the nlargest method
largest_df =  # YOUR CODE
# TODO: get the level 1 of the multiindex
index_to_keep =  # YOUR CODE

links_df = melted_df_sub.loc[index_to_keep]
links_df

We need to cast this data frame as a list of dictionaries as we have done for the list of nodes. 

>Use a similar code to than for the nodes to create a list of links: 

In [ ]:
# TODO: create the list of links
links_list =  # YOUR CODE
links_list

We now create the final dictionary for the network and save it into a json file

In [ ]:
network_dict = {"nodes": nodes_list,
                "links": links_list}

with open("./data/physicists.json","w") as f:
    json.dump(network_dict, f, indent=4)

If you have a mac, the following script is going to open a safari window to visualize this network, otherwise you can just open the index.html file with Safari or Firefox. For some reason it does not work with chrome.

In [ ]:
import os
os.system("open -a /Applications/Safari.app ./index.html")

Adjust the parameters to try to find nodes that tend to be grouped together. You can try to recreate the network with with different number of links.